In [1]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

# Ensures that all dataframes are displayed on one line instead of breaking columns across multiple lines
pd.set_option('display.expand_frame_repr', False)

In [2]:
def make_query(query, domain='https://vocab.getty.edu/sparql'):
    sparql = SPARQLWrapper(domain)
    sparql.setReturnFormat(JSON)

    sparql.setQuery(query)
    results = sparql.queryAndConvert()
    return pd.json_normalize(results['results']['bindings'])

## Simple Structure of a Query

In [3]:
query = '''
# Can be replaced with * to select all variables
SELECT ?subject ?predicate ?object
WHERE {
    ?subject ?predicate ?object .
}

OFFSET 10 LIMIT 10
'''

make_query(query)

,subject.type,subject.value,predicate.type,predicate.value,object.type,object.value
0,uri,http://www.ontotext.com/proton/protontop#type1,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
1,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
2,uri,http://www.ontotext.com/proton/protontop#premise,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
3,uri,http://www.w3.org/2002/07/owl#inverseOf,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
4,uri,http://www.ontotext.com/proton/protontop#premise1,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
5,uri,http://www.ontotext.com/proton/protontop#type2,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
6,uri,http://www.ontotext.com/proton/protontop#restr...,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
7,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#sub...,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
8,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#pre...,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
9,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#object,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...


## Prefixes

In [4]:
query = '''
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT *
WHERE {
    ?subject rdfs:label ?object .
}
LIMIT 10
'''

make_query(query)

,subject.type,subject.value,object.type,object.value
0,uri,http://vocab.getty.edu/ontology,literal,Getty Vocabulary Program ontology
1,uri,http://vocab.getty.edu/aat/,literal,Art and Architecture Thesaurus
2,uri,http://vocab.getty.edu/ulan/500279982,literal,aat
3,uri,http://vocab.getty.edu/tgn/,literal,Thesaurus of Geographic Names
4,uri,http://vocab.getty.edu/ulan/,literal,Union List of Artist Names
5,uri,http://www.getty.edu/research/,literal,Getty Research Institute
6,uri,http://www.ontotext.com,literal,Ontotext Corp
7,uri,http://vocab.getty.edu/ontology#Subject,literal,Subject
8,uri,http://vocab.getty.edu/ontology#broader,literal,broader
9,uri,http://vocab.getty.edu/ontology#broaderGeneric...,literal,broaderGenericExtended


## Shorthand Notation

In [5]:
query = '''
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX gvp: <http://vocab.getty.edu/ontology#>
PREFIX xl: <http://www.w3.org/2008/05/skos-xl#>

SELECT ?collaboratorName ?parentName
WHERE {
    ?person rdfs:label "Vincent van Gogh" .
    ?person gvp:ulan1303_collaborated_with ?collaborator .
    ?person gvp:ulan1511_child_of ?parent .

    ?collaborator gvp:prefLabelGVP ?collaboratorPrefLabel .
    ?collaboratorPrefLabel xl:literalForm ?collaboratorName .

    ?parent gvp:prefLabelGVP ?parentPrefLabel .
    ?parentPrefLabel xl:literalForm ?parentName .
}
LIMIT 10
'''

make_query(query)

,collaboratorName.xml:lang,collaboratorName.type,collaboratorName.value,parentName.type,parentName.value
0,en,literal,"Breitner, George Hendrik",literal,"Gogh-Carbentus, Anna Cornelia van"


In [6]:
query = '''
SELECT ?collaborator ?parent
WHERE {
    ?person rdfs:label "Vincent van Gogh" ;
            gvp:ulan1303_collaborated_with [ gvp:prefLabelGVP [ xl:literalForm ?collaborator ]] ;
            gvp:ulan1511_child_of [ gvp:prefLabelGVP [ xl:literalForm ?parent ]] .
}
LIMIT 10
'''

make_query(query)

,collaborator.xml:lang,collaborator.type,collaborator.value,parent.type,parent.value
0,en,literal,"Breitner, George Hendrik",literal,"Gogh-Carbentus, Anna Cornelia van"


## Optional Clauses

In [7]:
query = '''
SELECT ?artist ?artistName ?niecesNephews
WHERE {
    ?artist gvp:broaderInstantial ulan:500000002 ;
            gvp:prefLabelGVP [ xl:literalForm ?artistName ] .

    OPTIONAL {
        ?artist gvp:ulan1532_uncle-aunt_of ?niecesNephews .
    }
    
}
OFFSET 940 LIMIT 10
'''

make_query(query)

,artist.type,artist.value,artistName.xml:lang,artistName.type,artistName.value,niecesNephews.type,niecesNephews.value
0,uri,http://vocab.getty.edu/ulan/500029152,en,literal,"Acart, Adeline",NaN,NaN
1,uri,http://vocab.getty.edu/ulan/500226666,NaN,literal,"Acayaba, Carlos",NaN,NaN
2,uri,http://vocab.getty.edu/ulan/500103068,NaN,literal,"Acayaba, Marcos",NaN,NaN
3,uri,http://vocab.getty.edu/ulan/500246553,NaN,literal,"Acayaba, Marlene Milan",NaN,NaN
4,uri,http://vocab.getty.edu/ulan/500084911,NaN,literal,"Accama, Bernardus",uri,http://vocab.getty.edu/ulan/500433831
5,uri,http://vocab.getty.edu/ulan/500645759,NaN,literal,"Accama, Bernardus, II",NaN,NaN
6,uri,http://vocab.getty.edu/ulan/500133279,NaN,literal,"Accama, Mathys",NaN,NaN
7,uri,http://vocab.getty.edu/ulan/500589981,nl,literal,"Accama, Matthijs",NaN,NaN
8,uri,http://vocab.getty.edu/ulan/500433831,NaN,literal,"Accama, Simon",NaN,NaN
9,uri,http://vocab.getty.edu/ulan/500645762,nl,literal,"Accama, Symon",NaN,NaN


## Filters

In [8]:
query = '''
SELECT ?artist ?artistName ?birthYear ?deathYear
WHERE {
    ?artist gvp:broaderInstantial ulan:500000002 ;
            gvp:prefLabelGVP [ xl:literalForm ?artistName ] ;
            foaf:focus [ gvp:biographyPreferred [ gvp:estStart ?birthYear ; gvp:estEnd ?deathYear ]] .

    FILTER(YEAR(?birthYear) > 1800) # && YEAR(?birthYear) < 2024
    
}
ORDER BY ASC(?birthYear)
LIMIT 10
'''

make_query(query)

,artist.type,artist.value,artistName.type,artistName.value,birthYear.datatype,birthYear.type,birthYear.value,deathYear.datatype,deathYear.type,deathYear.value,artistName.xml:lang
0,uri,http://vocab.getty.edu/ulan/500089629,literal,"Adams, William",http://www.w3.org/2001/XMLSchema#gYear,literal,1801,http://www.w3.org/2001/XMLSchema#gYear,literal,1850,NaN
1,uri,http://vocab.getty.edu/ulan/500006886,literal,"Adam, Victor",http://www.w3.org/2001/XMLSchema#gYear,literal,1801,http://www.w3.org/2001/XMLSchema#gYear,literal,1866,nl
2,uri,http://vocab.getty.edu/ulan/500058753,literal,"Akroyd, James Lloyd",http://www.w3.org/2001/XMLSchema#gYear,literal,1801,http://www.w3.org/2001/XMLSchema#gYear,literal,1856,NaN
3,uri,http://vocab.getty.edu/ulan/500017852,literal,"Alexander, James E.",http://www.w3.org/2001/XMLSchema#gYear,literal,1801,http://www.w3.org/2001/XMLSchema#gYear,literal,1901,NaN
4,uri,http://vocab.getty.edu/ulan/500354680,literal,"Allsworth, William",http://www.w3.org/2001/XMLSchema#gYear,literal,1801,http://www.w3.org/2001/XMLSchema#gYear,literal,1866,NaN
5,uri,http://vocab.getty.edu/ulan/500028008,literal,"Amans, Jacques",http://www.w3.org/2001/XMLSchema#gYear,literal,1801,http://www.w3.org/2001/XMLSchema#gYear,literal,1890,nl
6,uri,http://vocab.getty.edu/ulan/500183457,literal,"Amic, Clarisse",http://www.w3.org/2001/XMLSchema#gYear,literal,1801,http://www.w3.org/2001/XMLSchema#gYear,literal,1951,NaN
7,uri,http://vocab.getty.edu/ulan/500010623,literal,"Andrews, James",http://www.w3.org/2001/XMLSchema#gYear,literal,1801,http://www.w3.org/2001/XMLSchema#gYear,literal,1876,nl
8,uri,http://vocab.getty.edu/ulan/500406311,literal,"Angelvy, G.",http://www.w3.org/2001/XMLSchema#gYear,literal,1801,http://www.w3.org/2001/XMLSchema#gYear,literal,1980,nl
9,uri,http://vocab.getty.edu/ulan/500502391,literal,"Anschütz, Gustav",http://www.w3.org/2001/XMLSchema#gYear,literal,1801,http://www.w3.org/2001/XMLSchema#gYear,literal,1901,NaN


## Values

In [9]:
query = '''
SELECT ?name ?person ?collaborator ?parent
WHERE {
    VALUES (?name) { ("Vincent van Gogh") ("Paul Klee") ("John Singer Sargent") }

    ?person rdfs:label ?name .

    OPTIONAL {
        ?person gvp:ulan1303_collaborated_with [ gvp:prefLabelGVP [ xl:literalForm ?collaborator ]] ;
                gvp:ulan1511_child_of [ gvp:prefLabelGVP [ xl:literalForm ?parent ]] .
    }
}
'''

make_query(query)

,name.type,name.value,person.type,person.value,collaborator.xml:lang,collaborator.type,collaborator.value,parent.type,parent.value
0,literal,Vincent van Gogh,uri,http://vocab.getty.edu/ulan/500115588,en,literal,"Breitner, George Hendrik",literal,"Gogh-Carbentus, Anna Cornelia van"
1,literal,Paul Klee,uri,http://vocab.getty.edu/ulan/500010493,NaN,NaN,NaN,NaN,NaN
2,literal,John Singer Sargent,uri,http://vocab.getty.edu/ulan/500023972,NaN,NaN,NaN,NaN,NaN


## Group by Clauses

In [10]:
query = '''
SELECT ?birthYear (COUNT(?artist) AS ?count)
WHERE {
    ?artist gvp:broaderInstantial ulan:500000002 ;
            gvp:prefLabelGVP [ xl:literalForm ?artistName ] ;
            foaf:focus [ gvp:biographyPreferred [ gvp:estStart ?birthYear ; gvp:estEnd ?deathYear ]] .
}
GROUP BY ?birthYear
ORDER BY DESC(?count)
LIMIT 10
'''

make_query(query)

,birthYear.datatype,birthYear.type,birthYear.value,count.datatype,count.type,count.value
0,http://www.w3.org/2001/XMLSchema#gYear,literal,1850,http://www.w3.org/2001/XMLSchema#integer,literal,29078
1,http://www.w3.org/2001/XMLSchema#gYear,literal,1920,http://www.w3.org/2001/XMLSchema#integer,literal,8616
2,http://www.w3.org/2001/XMLSchema#gYear,literal,1500,http://www.w3.org/2001/XMLSchema#integer,literal,5813
3,http://www.w3.org/2001/XMLSchema#gYear,literal,1940,http://www.w3.org/2001/XMLSchema#integer,literal,4712
4,http://www.w3.org/2001/XMLSchema#gYear,literal,1900,http://www.w3.org/2001/XMLSchema#integer,literal,3901
5,http://www.w3.org/2001/XMLSchema#gYear,literal,1930,http://www.w3.org/2001/XMLSchema#integer,literal,2840
6,http://www.w3.org/2001/XMLSchema#gYear,literal,1950,http://www.w3.org/2001/XMLSchema#integer,literal,2743
7,http://www.w3.org/2001/XMLSchema#gYear,literal,1871,http://www.w3.org/2001/XMLSchema#integer,literal,2462
8,http://www.w3.org/2001/XMLSchema#gYear,literal,1947,http://www.w3.org/2001/XMLSchema#integer,literal,2248
9,http://www.w3.org/2001/XMLSchema#gYear,literal,1952,http://www.w3.org/2001/XMLSchema#integer,literal,2193


## Federated Queries

In [11]:
query = '''
SELECT ?child ?childLabel
WHERE
{
    ?child wdt:P22 wd:Q1339 .

    SERVICE wikibase:label {
        bd:serviceParam wikibase:language "[AUTO_LANGUAGE]" .
    }
}
'''

make_query(query, domain='https://query.wikidata.org/sparql')

,child.type,child.value,childLabel.type,childLabel.value
0,uri,http://www.wikidata.org/entity/Q57225,literal,Q57225
1,uri,http://www.wikidata.org/entity/Q76428,literal,Q76428
2,uri,http://www.wikidata.org/entity/Q106641,literal,Q106641
3,uri,http://www.wikidata.org/entity/Q107277,literal,Q107277
4,uri,http://www.wikidata.org/entity/Q470198,literal,Q470198
5,uri,http://www.wikidata.org/entity/Q710996,literal,Q710996
6,uri,http://www.wikidata.org/entity/Q15079141,literal,Q15079141
7,uri,http://www.wikidata.org/entity/Q15735857,literal,Q15735857
8,uri,http://www.wikidata.org/entity/Q21029009,literal,Q21029009
9,uri,http://www.wikidata.org/entity/Q21029016,literal,Q21029016
